### Load the data

In [ ]:
import os
import mlflow

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
mlflow.set_experiment('Training_LSTM_RNN_Cluster')

### Create the Optuna Objective

In [ ]:
import mlflow.keras
import optuna
from common import OptunaPruneCallback, mean_absolute_percentage_error_keras, create_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import numpy as np


def objective(trial, data, coins):
    with mlflow.start_run() as run:
        mlflow.keras.autolog(log_models=False)

        # Define the search space for hyperparameters
        num_layers = trial.suggest_int('num_layers', 1, 4)
        units_per_layer = [trial.suggest_int(f'units_layer_{i}', 32, 256, 32) for i in range(num_layers)]
        sequence_length = trial.suggest_int('sequence_length', 1, 10)
        learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-3)
        dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
        min_max_scaling = trial.suggest_int('min_max_scaling', 0, 1)
        layer_type = trial.suggest_categorical('layer_type', ['LSTM', 'RNN'])

        layer_class = keras.layers.LSTM if layer_type == 'LSTM' else keras.layers.SimpleRNN 

        mlflow.log_params({
            'coin': ','.join(coins),
            'num_layers': num_layers,
            'units_per_layer': units_per_layer,
            'sequence_length': sequence_length,
            'learning_rate': learning_rate,
            'dropout_rate': dropout_rate,
            'min_max_scaling': min_max_scaling
        })

        if min_max_scaling == 1:
            scaler = MinMaxScaler()
            data = scaler.fit_transform(np.array(data))

        X, y = create_sequences(data, sequence_length)

        # Build and compile the LSTM model
        model = keras.Sequential()
        for units in units_per_layer[:-1]:
            model.add(layer_class(units, activation='relu', return_sequences=True, input_shape=(sequence_length, len(coins))))
            model.add(keras.layers.Dropout(dropout_rate))
        model.add(layer_class(units_per_layer[-1], activation='relu', input_shape=(sequence_length, len(coins))))
        model.add(keras.layers.Dropout(dropout_rate))
        model.add(keras.layers.Dense(len(coins)))

        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
        model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[mean_absolute_percentage_error_keras])
        # Split the data into training and validation sets
        _X_train, X_val, _y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
        X_train, X_test, y_train, y_test = train_test_split(_X_train, _y_train, test_size=0.2, shuffle=False)

        # Train the model with early stopping
        history = model.fit(
            X_train,
            y_train,
            epochs=50,
            batch_size=32,
            validation_data=(X_val, y_val), 
            verbose=0,
            callbacks=[OptunaPruneCallback(trial=trial)]
        )

        # Evaluate the model on the validation set
        loss = model.evaluate(X_val, y_val)

        return loss[1]

### Run the optimization

In [ ]:

import numpy as np

def evaluate_best_coin(coin, data, best_params):

    # Train the final model with the best hyperparameters
    # Define the search space for hyperparameters
    num_layers = best_params['num_layers']
    units_per_layer = [best_params[f'units_layer_{i}'] for i in range(num_layers)]
    sequence_length = best_params['sequence_length']
    learning_rate = best_params['learning_rate']
    dropout_rate = best_params['dropout_rate']
    min_max_scaling = best_params['min_max_scaling']
    layer_type = best_params['layer_type']

    layer_class = keras.layers.LSTM if layer_type == 'LSTM' else keras.layers.SimpleRNN 

    if min_max_scaling == 1:
        scaler = MinMaxScaler()
        data = scaler.fit_transform(np.array(data))

    X, y = create_sequences(data, sequence_length)
    _X_train, X_val, _y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(_X_train, _y_train, test_size=0.2, shuffle=False)
    # concatenate train and validation sets
    X_train = np.concatenate((X_train, X_val))
    y_train = np.concatenate((y_train, y_val))

    with mlflow.start_run(run_name=f"Training_best_model_{coin}") as run:
        # Build and compile the LSTM model
        best_model = keras.Sequential()
        for units in units_per_layer[:-1]:
            best_model.add(layer_class(units, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)))
            best_model.add(keras.layers.Dropout(dropout_rate))
        best_model.add(layer_class(units_per_layer[-1], activation='relu', input_shape=(sequence_length, 1)))
        best_model.add(keras.layers.Dropout(dropout_rate))
        best_model.add(keras.layers.Dense(1))

        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
        best_model.compile(optimizer=optimizer, loss='mean_squared_error')

        best_model.fit(X_train, y_train, epochs=100, batch_size=32)

        from common import register_training_experiment
        preds = best_model.predict([X_test])

        if min_max_scaling == 1:
            preds = scaler.inverse_transform(preds)
            y_test = scaler.inverse_transform(y_test)

        register_training_experiment(y_test, preds, model_name = layer_class, coin = coin)

In [ ]:
import numpy as np
from common import get_clustered_dataframes

# Normalize the data to the range [0, 1] to help the LSTM model converge faster

clusters_data = get_clustered_dataframes()

for cluster, data in clusters_data.items():
    print(f"Cluster {cluster}")
    
    data = data.drop(columns=['Date'])
    study_name = f"Cluster_{cluster}"  # Unique identifier of the study.
    storage_name = "sqlite:///training_cluster.db"
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    study = optuna.create_study(study_name=study_name, direction='minimize', storage=storage_name, pruner=pruner)
    criptos = data.columns

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, criptos), n_trials=50)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(criptos, data, best_params)
    except ValueError:
        print("No best hyperparameters found")